In [24]:
from openmm import app
from openmm import *
from openmm import unit
from openff.toolkit import Molecule, Topology
from openff.interchange import Interchange
from openmmforcefields.generators import (
    SMIRNOFFTemplateGenerator,
)
from openff.toolkit.typing.engines.smirnoff import ForceField
from sys import stdout

In [2]:
import pandas as pd
import ase
import ase.io as aio
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import rdDistGeom

In [3]:
df = pd.read_csv("./fda_approved_drugs.txt", sep="\t")
df = df.query("~smiles.isna()")
penicillin_smiles = df.query("generic_name == 'Penicillin G'")["smiles"].values[0]
penicillin_mol = Chem.MolFromSmiles(penicillin_smiles)

In [25]:
mol = Molecule.from_rdkit(penicillin_mol)
mol.generate_conformers(n_conformers=1)
top = Topology.from_molecules([mol])
ff = ForceField("openff-2.0.0.offxml")
interchange = Interchange.from_smirnoff(ff, top)

In [26]:
omm_system = interchange.to_openmm()
omm_top = interchange.to_openmm_topology()
omm_pos = interchange.positions.to_openmm()

In [30]:
integrator = LangevinIntegrator(300*unit.kelvin, 1/unit.picoseconds, 1*unit.femtoseconds)
simulation = app.Simulation(omm_top, omm_system, integrator)
simulation.context.setPositions(omm_pos)

In [32]:
simulation.minimizeEnergy()
simulation.reporters.append(app.StateDataReporter(stdout, 100, step=True, potentialEnergy=True, temperature=True))
simulation.reporters.append(app.PDBReporter("penicillin_trajectory.pdb", 100_000))
simulation.step(1_000_000)

50100,420.2556457519531,146.0352076343183
#"Step","Potential Energy (kJ/mole)","Temperature (K)"
50100,420.2556457519531,146.0352076343183
50200,438.1434326171875,135.31854663218007
50200,438.1434326171875,135.31854663218007
50300,431.904052734375,190.34730593206604
50300,431.904052734375,190.34730593206604
50400,448.8584289550781,153.37894757719852
50400,448.8584289550781,153.37894757719852
50500,451.64508056640625,196.1941583863871
50500,451.64508056640625,196.1941583863871
50600,441.491455078125,210.21274712323716
50600,441.491455078125,210.21274712323716
50700,452.64862060546875,195.0326059400263
50700,452.64862060546875,195.0326059400263
50800,443.8762512207031,261.6131237172329
50800,443.8762512207031,261.6131237172329
50900,455.078857421875,205.7668479348371
50900,455.078857421875,205.7668479348371
51000,455.383544921875,249.96893842721218
51000,455.383544921875,249.96893842721218
51100,451.4415283203125,241.58476183379048
51100,451.4415283203125,241.58476183379048
51200,447.406